In [1]:
import numpy as np
import pandas as pd
import ampligraph

In [2]:
ampligraph.__version__

'1.2.0'

In [3]:
df = pd.read_pickle("../Data/cleanDataset.pkl")

In [14]:
df["train"] = df.Sujeto > "S14"

In [15]:
df.train.value_counts()

True     22976801
False     8493802
Name: train, dtype: int64

In [16]:
df

,ECG,EMG,EDA,TEMP,RESP,Sujeto,label,train
64564,-0.014648,-0.001968,5.543137,32.567444,2.757263,S17,1,True
64565,-0.001236,-0.005539,5.542755,32.567444,2.742004,S17,1,True
64566,0.007278,0.017761,5.556107,32.561493,2.603149,S17,1,True
64567,0.009201,0.032181,5.558014,32.559998,2.749634,S17,1,True
64568,0.007141,-0.011032,5.564117,32.558502,2.760315,S17,1,True
...,...,...,...,...,...,...,...,...
4111478,-0.182602,-0.015793,0.484085,31.926239,-1.609802,S2,4,True
4111479,-0.170609,0.000687,0.473404,31.932190,-1.646423,S2,4,True
4111480,-0.160812,0.004532,0.463486,31.918823,-1.643372,S2,4,True
4111481,-0.156326,0.000595,0.459290,31.932190,-1.661682,S2,4,True


In [20]:
df["data_id"] = df.index.values.astype(str)
df["data_id"] = "Dato" + df.data_id
df["subject_id"] = df.Sujeto.values.astype(str)

In [ ]:
triples = []
for _, row in df[df["train"]].iterrows():
    # Data info
    ecg_triple = (row["ECG"], "isECGDataIn", row["data_id"])
    emg_triple = (row["EMG"], "isEMGDataIn", row["data_id"])
    eda_triple = (row["EDA"], "isEDADataIn", row["data_id"])
    temp_triple = (row["TEMP"], "isTEMPDataIn", row["data_id"])
    resp_triple = (row["RESP"], "isRESPDataIn", row["data_id"])
    
    # Stress results
    if row["label"] == 1:
        label_triple = (row["data_id"], "ShowsAStateOf", "Baseline")
    elif row["label"] == 2:
        label_triple = (row["data_id"], "FeelingOfData", "Stress")
    elif row["label"] == 3:
        label_triple = (row["data_id"], "FeelingOfData", "Amusement")
    elif row["label"] == 4:
        label_triple = (row["data_id"], "FeelingOfData", "Meditaion")
    
    suj_triple = (row["data_id"], "isProducedBy", row["subject_id"])
    
    triples.extend((ecg_triple, emg_triple, eda_triple, temp_triple, 
                    resp_triple, label_triple, suj_triple))

In [ ]:
triples_df = pd.DataFrame(triples, columns=["subject", "predicate", "object"])
triples_df[(triples_df.subject=="Dato64566") | (triples_df.object=="Dato64566")]

In [ ]:
from ampligraph.evaluation import train_test_split_no_unseen 

X_train, X_valid = train_test_split_no_unseen(np.array(triples), test_size=10000)
print('Train set size: ', X_train.shape)
print('Test set size: ', X_valid.shape)

In [ ]:
from ampligraph.latent_features import ComplEx

model = ComplEx(batches_count=50,
                epochs=300,
                k=100,
                eta=20,
                optimizer='adam', 
                optimizer_params={'lr':1e-4},
                loss='multiclass_nll',
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                seed=0, 
                verbose=True)

In [ ]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(X_train)

In [ ]:
from ampligraph.utils import save_model

save_model(model, model_name_path = "../Data/KGmodel.pkl")